<a href="https://colab.research.google.com/github/CesarPortales/BootCamp/blob/master/HADOOP_I_M%C3%ADo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HADOOP desde CERO

## Intro

No es más que una colección de herramientas open source (algunas hasta closed source).

![ML xkcd](https://imgs.xkcd.com/comics/machine_learning.png)

## Primero instalemos y probemos HADOOP 2.7.+


```
docker ps

docker run hello-world

docker pull sequenceiq/hadoop-docker:2.7.1

```
### Hasta aqui que ha pasado?

[A la pizarra!](https://witeboard.com/9734d460-7306-11e8-904a-07457219cdcb)  

[Mejor en G Draw](https://docs.google.com/drawings/d/1-OmMtvMg1STC2RIMlmL4gCxi6MB_3evlwsOhn4RgeVU)

http://merrigrove.blogspot.com/2015/10/visualizing-docker-containers-and-images.html

### Entremos en accion

```
docker run -p 50070:50070 -p 8088:8088 -it  sequenceiq/hadoop-docker:2.7.1 /bin/bash -bash

/etc/bootstrap.sh

ls

pwd

cat

tail

mkdir

```

NOTA: Si encuentras errores de este tipo:    
![docker error](https://i.imgur.com/F0lTXv5.jpg)  
Tras ejecutar `docker run`, debes intentar reiniciar docker, o cambiar el primer puerto del parametro `-p` porque seguramente exista otro servicio ejecutandose en esos puertos (como apache).  
Por ejemplo: `docker run -p 127.0.0.1:50050:50070 -p 127.0.0.1:9988:8088 -it  sequenceiq/hadoop-docker:2.7.1 /bin/bash -bash`


### Es HADOOP UNIX sin mas?

Hablemos de POSIX:

Es un estándar para sistemos operativos.

En Unix hay un Kernel, y alrededor tiene un shell, que es el que permite interactuar con el kernel.
Es un interfaz que permite interactuar con el sistema de ficheros.

El hdfs es una copia de ese shell


- **P**ortable **O**perating **S**ystem **I**nterface (UNI**X**)
- Creado por Richard Stallman
- *PORTABLE*
- HDFS no es POSIX

![posix](https://fossbytes.com/wp-content/uploads/2016/09/posix-ieee.jpg)


https://www.slideshare.net/mcsrivas/design-scale-and-performance-of-maprs-distribution-for-hadoop

**RAZON PRINCIPAL**: "A key one is that you can only add data to the end of the file, not seek to the middle and write things. You cannot seamlessly map code that assumes that all filesystems are Posix-compatible to HDFS."
  
    

## HDFS

![data store](http://csis.pace.edu/~marchese/CS865/Lectures/Chap7/Chapter7fin_files/image002.jpg)

### Que debe existir dentro de un HDFS?

- Ficheros inmutables (csv, txt, etc)
- Jars de mapreduce
- Librerias
- Ficheros de secuencia

Tendremos un data store distribuído en varios nodos

### Juguemos con HDFS

```
cd $HADOOP_PREFIX

bin/hdfs dfs -mkdir pruebas

bin/hdfs dfs -put README.txt pruebas/README.txt # Va a fallar si no existe la carpeta en hdfs

bin/hdfs dfs -ls pruebas

bin/hdfs dfs -touchz pruebas/test.zip

bin/hdfs dfs -ls pruebas

bin/hdfs dfs -cat pruebas/*

bin/hdfs dfs

jps  --> me dice qué servicios se están ejecutando en esta máquina. Viene de java virtual machine

permite hacer otro niveol de abstracción sobre los recursos. Permite a Java controlar los recursos sin pegarse con el SO.
ps --> todos los procesos que tengo corriendo (viene de powershell)
bin/hdfs dfs -du -h  --> información de mi unidad (sin el -h el espacio total, con el -h el espacio ocupado)
```


## HIGH AVAILABILITY
- FAULT TOLERANCE
- Resilient
- Elasticity


 ### JPS???
 
 Toda la arquitectura de hadoop está montada sobre java
 
 JPS es java processes
 
 ![jps arch](https://hadoop.apache.org/docs/r1.2.1/images/hdfsarchitecture.gif)

* HDFS tiene una arquitectura maestro/esclavo. 
* Un clúster HDFS consta de un único NameNode, un servidor maestro que administra el espacio de nombres del sistema de archivos y regula el acceso a los archivos por parte de los clientes. 
* Los DataNodes administran el almacenamiento.

### Arquitectura HDFS


Un datanode es un servicio que solo me da acceso a datos. Es una pieza de soft que me deja guardar o leer o borrar, ... sobre datos.

Los datanodes son como cajas en las que van los ficheros (o parte de los ficheros). Estas cajas no tienen inteligencia ninguan. Los gestiona un namenode, que es una máquina que sabe dónde están todos los datanodes.
El namenode tiene la ip, puerto, usuario, etc de cada datanode. Y no solo eso: también sabe dónde están las réplicas.

Cuando se ejecutan los comandos que hemos visto, se pueden ejecutar sobre el namenode o sobre el datanode. Si lo hago sobrer este, lo ejecuta sobre todos los datanodes. (es parecido al maestro y el esclavo).

Secundary namenode --> el hadoop obliga a levantarlo por si falla el datanode. Si este se cae, hadoop automáticamente levanta el secondary.

el namenode puede ser una máquina dedicada o no, y no necesita gran capacidad porque no almacena datos (los datos se guardan en los datanodes).

Cuando se trabaja en la nube, hablamos de máquinas gigantescas, con un software de virtualización que me hace creer que una parte de esa máquina es un ordenador (lo puedo hacer con docker o con lo que sea).

Los datanodes se pueden agrupar dentro de un rack. Ek rack es físico.
el client del esquema es, en nuestro caso la consola. Es un elemento de interacción con la arquitectura.

Hadoop lo inventó yahoo (cree). 

Hasta la ubicación física de los nodos es importantísima.

* Los NameNode y DataNode son piezas de software diseñadas para ejecutarse en máquinas básicas. 
* Estas máquinas suelen ejecutar un sistema operativo GNU / Linux. 
* HDFS está construido usando en Java.
* La existencia de un NameNode único en un clúster simplifica enormemente la arquitectura del sistema. 
* NameNode es el árbitro y repositorio de todos los metadatos HDFS. 
* El sistema está diseñado de tal manera que los datos del usuario nunca fluyan a través del NameNode.
* La ubicacion fisica de los nodos es de vital importancia y requiere mucha practica (ensayo y error).

### Mas lectura:

- https://hadoop.apache.org/docs/r1.2.1/hdfs_design.html
- https://fossbytes.com/posix-what-is-the-portable-operating-system-interface/


No estamos aún en Hadoop. 

El archivo bootstrap arranca el dfs y arranca el yarm. HDfs no es un sistema operativo. son varios ficheros ejecutables que lanzo y olvido

el HDFS  (Hadoop distributed file system)--> Han cogido el sistmea jerárquico de carpetas y archivos y lo han distribuído en varías máquinas.

Es una de las grandes herramientas que forman parte de Hadoop.

el hdfs me permite hacer todo esto con un cluster de Máquinas --> puedo tener 1, 2 20 máquinas que me soporten el sistema de ficheros.

HDFS distribuye la información por todos los nodos y hace un Data-store distribuído, y puedo tener 20 procesos leyendo esos datos en los 20 ficheros.

Para acceder al sistema de hadoop

**cd /usr/local/hadoop**

Si hago un ls, veo la carpeta bin, que tiene todos los comandos que puedo ejecutar en el HDFS.

Si quiero crear una carpeta  llamada pruebas

bin/hdfs dfs -mkdir pruebas

Esta carpeta la habrá creado dentro de /user/root. Para verla, no basta un simple ls

**bin/hdfs dfs -ls /user/root**
Así que estamos trabajando con dos niveles. Está por un lado el SO del container (un Unix) , y luego hay un nivel por encima que es el del sistema de ficheros distribuído.
este sistema solo está accesible a través de esta herramienta que es el DFS.

si quiero copiar un archivo, tengo que hacer un -put

bin/hdfs dfs -put README.txt pruebas/README.txt #el fichero README.txt exite en la carpeta  /usr/local/hadoop

Para ver que me lo ha copiado

bin/hdfs dfs -ls /user/root/pruebas


si quiero ver el contenido del archivo

**hdfs dfs -cat pruebas/README.txt**


Si quiero ver todo lo que se puede hacer:

 **bin/hdfs dfs**
 
 
 http://127.0.0.1:50070/dfshealth.html#tab-overview
 
el puerto 50070 es el puerto de HDFS.

Conesto accedo a toda la información de mi HDFS (Gigas libres, gigas ocupados,etc.)

si tuvier un cluster, en la pestaña Datanodes vería información 


![texto alternativo](https://)

el HDFS --> Han cogido el sistmea jerárquico de carpetas y archivos y lo han distribuído en varías máquinas.
Es una de las grandes herramientas que forman parte de Hadoop.

hasta ahora solo hemos visgto como meter ficheros en hdfs.
siempre que se instala haddop se introducen los examples. Es un java compilado con un montón de ejemplos de cosas que se pueden hacer.

## MAP REDUCE

### Juguemos con Map Reduce

Le vamos a pedir que se ejecute el jar de hadoop.  El comando grep busca texto dentro de una serie de ficheros. Nos vamos a meter dentro de la carpeta input y vamos abuscar un texto y el resultaod queremos que lo busque en la carpeta outpu

```
cd $HADOOP_PREFIX

bin/hadoop jar share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.1.jar grep input output 'dfs[a-z.]+'

```
El comando primero hace un mapeo, y debería haber escrito algo en la carpeta output.
Si ejecutamos:
bin/hdfs dfs -cat output/*
podremos ver qué ha creado
### Pero que ha pasado?

docker crea una red virtual. cuando estaba haciendo el mapeo, estaba cogiendo todas las líneas de los ficheros , busca todas las palabras y crea pares clave/ valor.
Luego hace el shuffing y lo reduce

Busquemos la IP del cluster, desde nuestro host:

```
docker network list

docker network inspect docker_default

```
Fijarse en gateway, y cambiar el `1` por un `2`.

Visitemos: http://127.0.0.1:50070 * NOTA - WINDOWS - En docker ANTES de hacer el run aparece el IP en verde, de esta manera:
![winip](https://i.imgur.com/Z8g9n4d.png)


### Que otra forma tendriamos de ver los resultados?


### Un poco de Teoria obligatoria

map: reading and formatting data
reduce: applying transformations/operations on all of the data

![mr](https://www.todaysoftmag.com/images/articles/tsm33/large/a11.png)


### Veamos el Codigo --> siemrpe tendrá dos partes, el map y el reduce

```java
import java.io.IOException;
import java.util.StringTokenizer;

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.Mapper;
import org.apache.hadoop.mapreduce.Reducer;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;

public class WordCount {

  public static class TokenizerMapper
       extends Mapper<Object, Text, Text, IntWritable>{

    private final static IntWritable one = new IntWritable(1);
    private Text word = new Text();

    public void map(Object key, Text value, Context context
                    ) throws IOException, InterruptedException {
      StringTokenizer itr = new StringTokenizer(value.toString());
      while (itr.hasMoreTokens()) {
        word.set(itr.nextToken());
        context.write(word, one);
      }
    }
  }

  public static class IntSumReducer
       extends Reducer<Text,IntWritable,Text,IntWritable> {
    private IntWritable result = new IntWritable();

    public void reduce(Text key, Iterable<IntWritable> values,
                       Context context
                       ) throws IOException, InterruptedException {
      int sum = 0;
      for (IntWritable val : values) {
        sum += val.get();
      }
      result.set(sum);
      context.write(key, result);
    }
  }

  public static void main(String[] args) throws Exception {
    Configuration conf = new Configuration();
    Job job = Job.getInstance(conf, "word count");
    job.setJarByClass(WordCount.class);
    job.setMapperClass(TokenizerMapper.class);
    job.setCombinerClass(IntSumReducer.class);
    job.setReducerClass(IntSumReducer.class);
    job.setOutputKeyClass(Text.class);
    job.setOutputValueClass(IntWritable.class);
    FileInputFormat.addInputPath(job, new Path(args[0]));
    FileOutputFormat.setOutputPath(job, new Path(args[1]));
    System.exit(job.waitForCompletion(true) ? 0 : 1);
  }
}

```


### Mas ejemplos

```
bin/hadoop jar share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.1.jar
```

### Mas lectura

- https://www.todaysoftmag.com/article/1358/hadoop-mapreduce-deep-diving-and-tuning